In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup 
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(r"C:\Users\Admin\Desktop\SOFTDEV2\SOFTWARE-DEVELOPMENT-2\ML\Data\chromedriver.exe")
driver.implicitly_wait(30)
driver.maximize_window()

def login():
    try:
        driver.get('https://www.finnomena.com/finno-login/')
        time.sleep(1)
        driver.find_element(By.NAME,'email').send_keys('bungaern2545@gmail.com')
        driver.find_element(By.CLASS_NAME,'custom-button').click()
        driver.find_element(By.NAME,'current_password').send_keys('tiwtiwtiw2545')
        driver.find_element(By.CLASS_NAME,'custom-button').click()
    except Exception as e:
        print(e)
login()

C:\Users\Admin\AppData\Local\Temp\ipykernel_2868\2100448294.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r"C:\Users\Admin\Desktop\SOFTDEV2\SOFTWARE-DEVELOPMENT-2\ML\Data\chromedriver.exe")


In [2]:
import pandas as pd

def getStockDetails(name,isAnnual):
    sections = ['stock-financial-report','stock-financial-ratio']
    url = 'https://www.finnomena.com/stock/' + name
    driver.get(url)
    quarters = []
    data = []
    toggled = False
    for section in sections:
            obj = {}
            keys = []
            headerElements = driver.find_element(By.CSS_SELECTOR,
        f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.topic-wrapper.user-select-none.float-left.overflow-shadow')
            if not toggled and isAnnual:
                toggled = True
                e = driver.find_element(By.CLASS_NAME,'toggle')
                webdriver.ActionChains(driver).move_to_element(e).click(e).perform()
            for topic in headerElements.find_elements(By.CLASS_NAME,'topic')[1:]:
                key = BeautifulSoup(topic.get_attribute('id')).get_text()
                keys.append(key)
                
            contentElements = driver.find_element(By.CSS_SELECTOR, 
                                f'#{section}>div>div>div>div>div.table-overflow-wrapper>div.content-wrapper.user-select-none')
            contentHTML = BeautifulSoup(contentElements.get_attribute('innerHTML'), 'html.parser')
            dataWrapper = contentHTML.find_all('div', {'class': 'data-wrapper'})
            
            if len(quarters) == 0:
                for div in contentHTML.find_all('div', {'class': 'year'}):
                    quarters.append(div.get_text())
            
            for i in range(len(dataWrapper)): 
                values = [data.get_text() for data in dataWrapper[i].find_all('div', {'class': 'data-each'})]
                key = f"{keys[i]}"
                obj[key] = values
                data.append(obj)
    return (quarters,data)

def getFinancialDetails(symbol,isAnnual):
    quarterlyOrAnnual,responseData = getStockDetails(name=symbol,isAnnual=isAnnual)
    data = {}
    for d in responseData:
        data = dict(list(data.items()) + list(d.items()))
    data['Period'] = quarterlyOrAnnual
    data['Symbol'] = symbol
    df = pd.DataFrame(data) 
    df = df.iloc[:-1]
    df = df.drop(df.columns[[3,5,6,8,9,10,11,12,15,16,17,18,19]], axis=1)
    return df

In [16]:
import sqlite3

conn = sqlite3.connect('share_V2.sqlite')
cursor = conn.cursor()
cursor.execute("""SELECT I.Symbol 
                    FROM Information as I
                    INNER JOIN Market as M ON I.MarketId = M.MarketId
                    WHERE Mname = 'SET'
                    ORDER BY Symbol ASC""")
result = cursor.fetchall()
values = [item[0][:-3] for item in result]
values

['2S',
 '3K-BAT',
 '7UP',
 'A',
 'AAI',
 'AAV',
 'ACC',
 'ACE',
 'ACG',
 'ADVANC',
 'AEONTS',
 'AFC',
 'AGE',
 'AH',
 'AHC',
 'AI',
 'AIE',
 'AIMCG',
 'AIMIRT',
 'AIT',
 'AJ',
 'AJA',
 'AKR',
 'ALLA',
 'ALLY',
 'ALT',
 'ALUCON',
 'AMARIN',
 'AMATA',
 'AMATAR',
 'AMATAV',
 'AMC',
 'AMR',
 'ANAN',
 'AOT',
 'AP',
 'APCO',
 'APCS',
 'APEX',
 'APURE',
 'AQ',
 'AQUA',
 'AS',
 'ASEFA',
 'ASIA',
 'ASIAN',
 'ASIMAR',
 'ASW',
 'AURA',
 'AWC',
 'B-WORK',
 'B',
 'B52',
 'BA',
 'BAFS',
 'BAM',
 'BANPU',
 'BAREIT',
 'BBGI',
 'BBL',
 'BCH',
 'BCP',
 'BCPG',
 'BCT',
 'BDMS',
 'BEAUTY',
 'BEC',
 'BEM',
 'BEYOND',
 'BGC',
 'BGRIM',
 'BH',
 'BIG',
 'BIOTEC',
 'BIZ',
 'BJC',
 'BJCHI',
 'BKD',
 'BKKCP',
 'BLA',
 'BLAND',
 'BLISS',
 'BOFFICE',
 'BPP',
 'BR',
 'BRI',
 'BROCK',
 'BRR',
 'BRRGIF',
 'BSBM',
 'BTG',
 'BTNC',
 'BTS',
 'BTSGIF',
 'BWG',
 'CBG',
 'CCET',
 'CCP',
 'CEN',
 'CENTEL',
 'CFRESH',
 'CGD',
 'CH',
 'CHG',
 'CHOTI',
 'CI',
 'CITY',
 'CIVIL',
 'CK',
 'CKP',
 'CM',
 'CMAN',
 'CMC',
 'CMR',
 '

In [17]:
import requests

nameNoData = ['AURA', 'COMM', 'TRUE']
details = []
for name in values:
    response = requests.get('https://www.finnomena.com/stock/' + name)
    if (response.status_code == 404) or (name in nameNoData):
        continue
    elif len(details) == 0:
        df1 = getFinancialDetails(symbol=name, isAnnual=False)
        details.append(df1)
    else:
        df2 = getFinancialDetails(symbol=name, isAnnual=False)
        df_old = details[0]
        df_new = pd.concat([df_old, df2])
        details[0] = df_new

In [18]:
details[0]

,Asset,TotalDebt,Equity,PaidUpCapital,Revenue,NetProfit,EarningPerShare,ROA,ROE,Period,Symbol
0,"1,169.46",508.33,661.13,250.00,"1,083.73",32.00,0.13,2.64,4.97,1Q2556,2S
1,982.97,319.63,663.34,300.00,940.55,7.77,0.00,0.72,1.17,2Q2556,2S
2,"1,338.60",654.82,683.78,300.00,"1,092.56",20.44,0.07,1.76,3.03,3Q2556,2S
3,"1,222.08",531.40,690.69,300.00,983.91,7.83,0.03,0.61,1.14,4Q2556,2S
4,"1,355.38",629.06,726.32,300.00,"1,086.03",32.09,0.11,2.49,4.53,1Q2557,2S
...,...,...,...,...,...,...,...,...,...,...,...
14,"2,557.20","1,397.83","1,159.37",300.00,780.53,10.06,0.03,0.40,0.87,4Q2564,ZEN
15,"2,559.30","1,379.12","1,180.18",300.00,692.19,8.12,0.03,0.32,0.69,1Q2565,ZEN
16,"2,591.15","1,361.71","1,229.44",300.00,864.68,49.26,0.16,1.91,4.09,2Q2565,ZEN
17,"2,703.67","1,418.70","1,284.97",300.00,910.70,54.18,0.18,2.05,4.31,3Q2565,ZEN


In [20]:
# details[0].to_csv('finnomena.csv', index=False)

In [5]:
detail = getFinancialDetails(symbol='2S', isAnnual=False)
detail

,Asset,TotalDebt,Equity,Revenue,NetProfit,ROA,ROE,Period,Symbol
0,"1,169.46",508.33,661.13,"1,083.73",32.00,2.64,4.97,1Q2556,2S
1,982.97,319.63,663.34,940.55,7.77,0.72,1.17,2Q2556,2S
2,"1,338.60",654.82,683.78,"1,092.56",20.44,1.76,3.03,3Q2556,2S
3,"1,222.08",531.40,690.69,983.91,7.83,0.61,1.14,4Q2556,2S
4,"1,355.38",629.06,726.32,"1,086.03",32.09,2.49,4.53,1Q2557,2S
5,"1,345.54",636.41,709.13,"1,101.62",18.81,1.39,2.62,2Q2557,2S
6,"1,391.75",680.79,710.95,"1,063.94",1.82,0.13,0.26,3Q2557,2S
7,"1,207.02",506.88,700.13,995.07,-10.82,-0.83,-1.53,4Q2557,2S
8,"1,217.56",456.53,761.03,"1,035.84",1.57,0.13,0.22,1Q2558,2S
9,"1,375.11",617.59,757.52,925.63,26.49,2.04,3.49,2Q2558,2S
